In [1]:
import cv2
import numpy as np
import math
from IPython.display import Image
from matplotlib import pyplot as plt
from jetbot import Camera, bgr8_to_jpeg
from IPython.display import display
import ipywidgets.widgets as widgets
import traitlets
import os
from uuid import uuid1
from IPython.display import clear_output
from jetbot import Robot
import array as arr
import time
%matplotlib inline

In [2]:
from jetbot import Robot
import time

In [2]:
def show_image(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def gaussain_detection_blur(frame):
    # Gaussian blur
    blur = cv2.GaussianBlur(frame, (5, 5), 15)
    #show_image("blur", blur)
    return blur

In [4]:
def detect_edges(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #show_image("hsv", hsv)
    # Once the image is in HSV, we can “lift” all the whitish colors from the image. This is by specifying a range of the color White
    # in HSV space. The range is [0,0,0] to [180,255,30]. This is because the range of values for Hue is [0,180] and Saturation and Value are [0,255].
    # The lower and upper bounds of the range are specified as NumPy arrays.
    sensitivity = 90
    lower_white = np.array([0,0,255-sensitivity])
    upper_white = np.array([255,sensitivity,255])
    mask = cv2.inRange(hsv, lower_white, upper_white)
    #show_image("mask", mask)

    # detect edges
    edges = cv2.Canny(mask, 200, 400)

    return edges

In [5]:
def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)

    # only focus bottom half of the screen
    polygon = np.array([[
        (0, height * 1 / 2),
        (width, height * 1 / 2),
        (width, height),
        (0, height),
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    cropped_edges = cv2.bitwise_and(edges, mask)
    #show_image("edges", cropped_edges)
    return cropped_edges

In [6]:
def display_lines(frame, line_segments):
    line_image = np.zeros_like(frame)
    if line_segments is not None:
        for line_segment in line_segments:
            for x1, y1, x2, y2 in line_segment:
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    line_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return line_image

In [7]:
def detect_line_segments(cropped_edges, frame):
    # tuning min_threshold, minLineLength, maxLineGap is a trial and error process by hand
    rho = 1  # distance precision in pixel, i.e. 1 pixel
    angle = np.pi / 180  # angular precision in radian, i.e. 1 degree
    min_threshold = 10  # minimal of votes
    line_segments = cv2.HoughLinesP(cropped_edges, rho, angle, min_threshold, 
                                    np.array([]), minLineLength=8, maxLineGap=4)

    # Display the line segments on the original image
    line_image = display_lines(frame, line_segments)
    #show_image("line segments", line_image)

    return line_segments

In [8]:
def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    y1 = height  # bottom of the frame
    y2 = int(y1 * 1 / 2)  # make points from middle of the frame down

    # bound the coordinates within the frame
    x1 = max(-width, min(2 * width, int((y1 - intercept) / slope)))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / slope)))
    return [[x1, y1, x2, y2]]

In [9]:
def average_slope_intercept(frame, line_segments):
    """
    This function combines line segments into one or two lane lines
    If all line slopes are < 0: then we only have detected left lane
    If all line slopes are > 0: then we only have detected right lane
    """
    lane_lines = []
    if line_segments is None:
        #print("No line_segment segments detected")
        return lane_lines

    height, width, _ = frame.shape
    left_fit = []
    right_fit = []

    boundary = 1/3
    left_region_boundary = width * (1 - boundary)  # left lane line segment should be on left 2/3 of the screen
    right_region_boundary = width * boundary # right lane line segment should be on left 2/3 of the screen

    for line_segment in line_segments:
        for x1, y1, x2, y2 in line_segment:
            if x1 == x2:
                #print('skipping vertical line segment (slope=inf): %s' % line_segment)
                continue
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < 0:
                if x1 < left_region_boundary and x2 < left_region_boundary:
                    left_fit.append((slope, intercept))
            else:
                if x1 > right_region_boundary and x2 > right_region_boundary:
                    right_fit.append((slope, intercept))

    left_fit_average = np.average(left_fit, axis=0)
    if len(left_fit) > 0:
        lane_lines.append(make_points(frame, left_fit_average))

    right_fit_average = np.average(right_fit, axis=0)
    if len(right_fit) > 0:
        lane_lines.append(make_points(frame, right_fit_average))

    #print('lane lines: %s' % lane_lines) # [[[316, 720, 484, 432]], [[1009, 720, 718, 432]]]

    return lane_lines

In [10]:
def compute_steering_angle(frame, lane_lines):
    """ Find the steering angle based on lane line coordinate
        We assume that camera is calibrated to point to dead center
    """
    if len(lane_lines) == 0:
        #print('No lane lines detected, do nothing')
        return -90

    height, width, _ = frame.shape
    if len(lane_lines) == 1:
        #print('Only detected one lane line, just follow it. %s' % lane_lines[0])
        x1, _, x2, _ = lane_lines[0][0]
        x_offset = x2 - x1
    else:
        _, _, left_x2, _ = lane_lines[0][0]
        _, _, right_x2, _ = lane_lines[1][0]
        camera_mid_offset_percent = 0.02 # 0.0 means car pointing to center, -0.03: car is centered to left, +0.03 means car pointing to right
        mid = int(width / 2 * (1 + camera_mid_offset_percent))
        x_offset = (left_x2 + right_x2) / 2 - mid

    # find the steering angle, which is angle between navigation direction to end of center line
    y_offset = int(height / 2)

    angle_to_mid_radian = math.atan(x_offset / y_offset)  # angle (in radian) to center vertical line
    angle_to_mid_deg = int(angle_to_mid_radian * 180.0 / math.pi)  # angle (in degrees) to center vertical line
    steering_angle = angle_to_mid_deg + 90  # this is the steering angle needed by picar front wheel

    #print('new steering angle: %s' % steering_angle)
    return steering_angle

In [11]:
def stabilize_steering_angle(curr_steering_angle, new_steering_angle, num_of_lane_lines, max_angle_deviation_two_lines=5, max_angle_deviation_one_lane=1):
    """
    Using last steering angle to stabilize the steering angle
    This can be improved to use last N angles, etc
    if new angle is too different from current angle, only turn by max_angle_deviation degrees
    """
    if num_of_lane_lines == 2 :
        # if both lane lines detected, then we can deviate more
        max_angle_deviation = max_angle_deviation_two_lines
    else :
        # if only one lane detected, don't deviate too much
        max_angle_deviation = max_angle_deviation_one_lane
    
    angle_deviation = new_steering_angle - curr_steering_angle
    if abs(angle_deviation) > max_angle_deviation:
        stabilized_steering_angle = int(curr_steering_angle
                                        + max_angle_deviation * angle_deviation / abs(angle_deviation))
    else:
        stabilized_steering_angle = new_steering_angle
    print('Proposed angle: %s, stabilized angle: %s' % (new_steering_angle, stabilized_steering_angle))
    return stabilized_steering_angle

In [12]:
def display_heading_line(frame, steering_angle, line_color=(0, 0, 255), line_width=5, ):
    heading_image = np.zeros_like(frame)
    height, width, _ = frame.shape

    # figure out the heading line from steering angle
    # heading line (x1,y1) is always center bottom of the screen
    # (x2, y2) requires a bit of trigonometry

    # Note: the steering angle of:
    # 0-89 degree: turn left
    # 90 degree: going straight
    # 91-180 degree: turn right 
    steering_angle_radian = steering_angle / 180.0 * math.pi
    x1 = int(width / 2)
    y1 = height
    x2 = int(x1 - height / 2 / math.tan(steering_angle_radian))
    y2 = int(height / 2)

    cv2.line(heading_image, (x1, y1), (x2, y2), line_color, line_width)
    heading_image = cv2.addWeighted(frame, 0.8, heading_image, 1, 1)

    return heading_image

In [13]:
def moveRobot(angle):
    motorSpeedDiff = 0.05
    straightSpeedLeft = 0.3 #0.08
    straightSpeedRight = 0.295
    straightSpeed = 0.3 #0.2
    radius = 0.0325
    L = 0.13
    
    deltaAngle = angle - 90
    angleRad = abs(deltaAngle * (math.pi/180))
    if deltaAngle < 0:
        leftWheel = ((straightSpeed - (L/2)*angleRad)/radius)*radius
        rightWheel = ((straightSpeed + (L/2)*angleRad)/radius)*radius
    if deltaAngle > 0:
        leftWheel = ((straightSpeed + (L/2)*angleRad)/radius)*radius
        rightWheel = ((straightSpeed - (L/2)*angleRad)/radius)*radius
    if deltaAngle == 0:
        leftWheel = straightSpeed
        rightWheel = straightSpeed
    
    finalLeft = (leftWheel)
    finalRight = (rightWheel-0.005)
    return [finalLeft, finalRight]


In [26]:
# def main():
    
#     count = 0
    
#     while (count < 200):
        
#         #print(count)
#         camera = Camera.instance(width=224, height=224)
#         image = widgets.Image(format='jpeg', width=224, height=224)

#         camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
        
#         image_path = os.path.join('sample.jpg')
#         with open(image_path, 'wb') as f:
#             f.write(image.value)
            
#         count+=1
    
#         #clear_output(wait=True)
#         #display(heading_image)
# #         plt.imshow(heading_image, interpolation='nearest')
# #         plt.show()

In [14]:
robot = Robot()
camera = Camera()

In [15]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    
    frame = image

    blur = gaussain_detection_blur(frame)

    edges = detect_edges(blur)

    cropped_edges = region_of_interest(edges)

    line_segments = detect_line_segments(cropped_edges, blur)

    lane_lines = average_slope_intercept(blur, line_segments)

    line_image = display_lines(blur, lane_lines)
        
    # Compute steering angle
    steering_angle = compute_steering_angle(blur, lane_lines)

    # Stabilize steering angle
    curr_steering_angle = 90
    stabilized_steering_angle = stabilize_steering_angle(curr_steering_angle, steering_angle, len(lane_lines))
    
    heading_image = display_heading_line(line_image, stabilized_steering_angle)
    
    if (stabilized_steering_angle < 90):
        stabilized_steering_angle -= 5
        heading_image = display_heading_line(line_image, stabilized_steering_angle)
    elif (stabilized_steering_angle > 90):
        stabilized_steering_angle += 5
        heading_image = display_heading_line(line_image, stabilized_steering_angle)

    # Show the steering angle on screen
    
    arrSpeeds = moveRobot(stabilized_steering_angle)
    print(arrSpeeds[0])
    print(arrSpeeds[1])
    robot.set_motors(arrSpeeds[0], arrSpeeds[1])
        
    #cv2.imwrite('Images/' + str(count) + '.jpg', heading_image)
    
    
    jpeg_image = bgr8_to_jpeg(heading_image)
    
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]))

Proposed angle: 54, stabilized angle: 89
0.2931932159172221
0.3018067840827779


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Proposed angle: 135, stabilized angle: 95
0.31134464013796315
0.2836553598620368
Proposed angle: 37, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 55, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 46, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 52, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 52, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 65, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 58, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 150, stabilized angle: 95
0.31134464013796315
0.2836553598620368
Proposed angle: 75, stabilized angle: 89
0.2931932159172221
0.3018067840827779
Proposed angle: 150, stabilized angle: 95
0.31134464013796315
0.2836553598620368
Proposed angle: 149, stabilized angle: 95
0.31134464013796315
0.2836553598620368
Proposed angle: 146, stabilized angle: 95
0.

In [22]:
robot.stop()